# Bee Detection

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import os
os.chdir("./drive/MyDrive")

##Create paths and determine model to be used

See [TensorFlow Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)

In [ ]:
CUSTOM_MODEL_NAME = 'efficientdet_d1_coco17_tpu-32_640x640_wo_augment' 
PRETRAINED_MODEL_NAME = "efficientdet_d1_coco17_tpu-32" 
BASE_MODEL_ZOO_URL = "http://download.tensorflow.org/models/object_detection/tf2/20200711"
PRETRAINED_MODEL_URL = os.path.join(BASE_MODEL_ZOO_URL, PRETRAINED_MODEL_NAME + ".tar.gz") 
LABEL_MAP_NAME = 'label_map.pbtxt'
TRAIN_TFRECORD_NAME = 'synthetic_train.record'
VALID_TFRECORD_NAME = 'synthetic_valid.record'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'DATA_PATH': os.path.join('Tensorflow', 'workspace','data'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CUSTOM_MODEL_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['DATA_PATH'], LABEL_MAP_NAME),
    'TRAIN_TFRECORD': os.path.join(paths['DATA_PATH'], TRAIN_TFRECORD_NAME),
    'VALID_TFRECORD': os.path.join(paths['DATA_PATH'], VALID_TFRECORD_NAME),
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

##Install TensorFlow Object Detection API

In [ ]:
if os.name=='nt':
    !pip install wget
    import wget

In [ ]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

In [ ]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
!pip install tensorflow --upgrade --quiet

In [ ]:
#!pip uninstall protobuf matplotlib -y 
#!pip install protobuf matplotlib==3.2 

##Load pretrained model for transer learning 

In [ ]:
import object_detection

In [ ]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

**Create Label Map**
not needed when already done

In [ ]:
labels = [{'name':'bee', 'id':1}]
if not os.path.exists(files['LABELMAP']):
  with open(files['LABELMAP'], 'w') as f:
      for label in labels:
          f.write('item { \n')
          f.write('\tname:\'{}\'\n'.format(label['name']))
          f.write('\tid:{}\n'.format(label['id']))
          f.write('}\n')

**Copy Model Config to Training Folder**

In [ ]:
if os.name =='posix' and not os.path.exists(files['PIPELINE_CONFIG']): #check if config was added manually
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {paths['CUSTOM_MODEL_PATH']}
if os.name == 'nt' and not os.path.exists(files['PIPELINE_CONFIG']):
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {paths['CUSTOM_MODEL_PATH']}

**Update Config For Transfer Learning**

See [Neptune TFOD Config](https://neptune.ai/blog/tensorflow-object-detection-api-best-practices-to-training-evaluation-deployment) for config explanations

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
files['PIPELINE_CONFIG']

In [ ]:
default_config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
default_config.model.ssd.loss.classification_weight = 0.0 #We do not care for the classification as there is only one class anyway

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [ ]:
continue_train = False #Continue training and use full model or transfer learn and use pretrained model without head

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels) #1 (i.e. bee)
#pipeline_config.model.ssd.image_resizer.keep_aspect_ratio_resizer.convert_to_grayscale = True #grayscaling

#TRAINING CONFIG
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.num_steps = 30000
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = 30000



if continue_train:
  pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['CUSTOM_MODEL_PATH'], 'ckpt-11')
  pipeline_config.train_config.load_all_detection_checkpoint_vars = True # load also the head variables
  pipeline_config.train_config.fine_tune_checkpoint_type = "full"
else:
  pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
  pipeline_config.train_config.fine_tune_checkpoint_type = "detection" #Restores the entire feature extractor. The only parts of the full detection model that are not restored are the box and class prediction heads.""

pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [files['TRAIN_TFRECORD']]

#EVALUATION CONFIG
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [files['VALID_TFRECORD']] 

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [ ]:
config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="Tensorflow/workspace/models/efficientdet_d1_coco17_tpu-32_640x640_wo_augment"

##Train the model

In [ ]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, paths['CUSTOM_MODEL_PATH'],files['PIPELINE_CONFIG'])

In [ ]:
print(command)

In [ ]:
!{command}

##Load trained model to make example prediction

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CUSTOM_MODEL_PATH'], 'ckpt-11')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], '10.jpg')

In [ ]:
img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

print(num_detections)

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=15,
            min_score_thresh=.0000001,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()